## Zadanie 2.1

In [2]:
import cv2
import numpy as np

In [3]:
def zadanie2(name, speed, threshold_value, kernel_1, kernel_2, blur):    
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()

    I_last = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    I_last = cv2.cvtColor(I_last,cv2.COLOR_BGR2GRAY)
    print(I_last.shape)
    #I_last = I_last.astype(int)

    for i in range (roi_start,roi_end, speed):
        #Load new frame
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I,cv2.COLOR_BGR2GRAY)
        #I = I.astype(int)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        #Image difference
        I_diff = cv2.absdiff(I,I_last)
        #I_diff = I_diff.astype('uint8')
        I_last = I

        #Filtration
        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)
        
        #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [5]:
zadanie2("highway", 5, 36, 3, 5, 11)

(240, 320)
P =  0.7926574423480084
R =  0.868903047177989
F1 =  0.8290308675081424


In [98]:
zadanie2("office", 5, 3, 5, 7, 15)

P =  0.7276996746270489
R =  0.3292694148223552
F1 =  0.45338931558667206


In [9]:
zadanie2("pedestrian", 5, 16, 3, 4, 11)

(240, 360)
P =  0.44410725074479684
R =  0.9572284963739643
F1 =  0.6067241440650831
